# Mask R-CNN - Train on Shapes Dataset


This notebook shows how to train Mask R-CNN on your own dataset. To keep things simple we use a synthetic dataset of shapes (squares, triangles, and circles) which enables fast training. You'd still need a GPU, though, because the network backbone is a Resnet101, which would be too slow to train on a CPU. On a GPU, you can start to get okay-ish results in a few minutes, and good results in less than an hour.

The code of the *Shapes* dataset is included below. It generates images on the fly, so it doesn't require downloading any data. And it can generate images of any size, so we pick a small image size to train faster. 

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


ModuleNotFoundError: No module named 'cv2'

## Configurations

In [ ]:
class HandsConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "HandsMasksOnly"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 16

    MEAN_PIXEL = np.array([123.7, 116.8, 103.9, 103.9]) # for RGB-D
    IMAGE_CHANNEL_COUNT = 4 

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 3 classes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 256 
    IMAGE_MAX_DIM = 320
    
    USE_MINI_MASK = False
    MINI_MASK_SHAPE = (64, 80)
    MASK_SHAPE = [28, 28] # SHOULD BE CHANGED also somewhere else
    IMAGE_RESIZE_MODE = "deform" # CUSTOM
    
    #BACKBONE = "resnet50"

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    TRAIN_ROIS_PER_IMAGE = 10

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = HandsConfig()
#config.display()

## Notebook Preferences

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Dataset

Create a synthetic dataset

Extend the Dataset class and add a method to load the shapes dataset, `load_shapes()`, and override the following methods:

* load_image()
* load_mask()
* image_reference()

In [ ]:
import skimage.io
class HandsDataset(utils.Dataset):

    # overriden for RGB-D
    def load_image(self, image_id):
        image_depth = skimage.io.imread(self.image_info[image_id]['path'])
        image_depth = image_depth[..., np.newaxis]
        image_rgb = skimage.io.imread(self.image_info[image_id]['path'].replace("depth","color"))
        # if image.ndim != 4: # TODO CHANGE WHEN INPUTS ARE GRAYSCALE/ 4channel
            # image = np.stack((image[:,:,0],)*4, axis=-1)
        image_full = np.concatenate([image_rgb, image_depth], axis=2)
        return image_full

    def load_dataset(self, dataset_dir, subset_suffix, config):
        self.config = config
        #self.add_class("hand", 1, "arm")
        self.add_class("hand", 1, "hand")
        #self.add_class("hand", 3, "fingertip")

        dataset_dir = os.path.join(dataset_dir, "depth"+subset_suffix)
        image_ids = list(os.walk(dataset_dir))[0][2]

        # Add images
        for image_id in image_ids:
            self.add_image(
                "hand",
                image_id=image_id,
                path=os.path.join(dataset_dir, image_id),
                subset = subset_suffix)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        info = self.image_info[image_id]

        # mask_full = []
        # for i in range(1,self.config.NUM_CLASSES):
        #     mask_couter = str(i) if i > 0 else ""
        #     mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "mask"+ mask_couter +info['subset'])
        #     mask_file_name = info['id'].replace("depth", "mask" + mask_couter)
        #     mask = skimage.io.imread(os.path.join(mask_dir, mask_file_name)).astype(np.bool)[:,:,0] # TODO CHANGE WHEN MASKS ARE GRAYSCALE !!! 
        #     mask_full.append(mask)
        # mask_full = np.stack(mask_full, axis=2)

        # class_ids = np.array(list(range(1, self.config.NUM_CLASSES)))
        #return mask_full, class_ids
        
        mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "mask2"+info['subset'])
        mask = []
        mask = skimage.io.imread(os.path.join(mask_dir, info['id'].replace("depth", "mask2"))).astype(np.bool)
        mask = mask[..., np.newaxis]
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID, we return an array of ones
        return mask, np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "hand":
            return info["id"]
        else:
            super(self.__class__, self).image_reference(image_id)
            
#DATASET_DIR = os.path.join(ROOT_DIR, "datasets", "rgbd_joined_dataset", "ruka_2")
DATASET_DIR = os.path.join("D:/", "datasets","blurred","3800")
# Training dataset
dataset_train = HandsDataset()
dataset_train.load_dataset(DATASET_DIR,"", config)
dataset_train.prepare()

# Validation dataset
dataset_val = HandsDataset()
#dataset_val.load_dataset(DATASET_DIR,"-valid")
dataset_val.load_dataset(DATASET_DIR,"", config)
dataset_val.prepare()

image_ids = np.random.choice(dataset_train.image_ids, 3)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names, limit = config.NUM_CLASSES - 1)

## Create Model

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last, last_no_bbox

# TODO copy weights for 4th channel - check https://github.com/orestis-z/mask-rcnn-rgbd/blob/d590e0f5085f8cbe895a6698e284426fd0116aa4/instance_segmentation/sceneNet/train.py#L66-L85

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    import copy
    config_rgb = copy.deepcopy(config)
    config_rgb.MEAN_PIXEL = config_rgb.MEAN_PIXEL[0:2]
    config_rgb.IMAGE_SHAPE = np.array([config.IMAGE_MAX_DIM, config.IMAGE_MAX_DIM, 3])
    model_rgb = modellib.MaskRCNN(mode="training", config=config_rgb, model_dir=MODEL_DIR)
    
    # Local path to trained weights file
    COCO_MODEL_PATH = os.path.join(ROOT_DIR, "Mask_RCNN-tf2/mask_rcnn_coco.h5")
    # Download COCO trained weights from Releases if needed
    if not os.path.exists(COCO_MODEL_PATH):
        utils.download_trained_weights(COCO_MODEL_PATH)
    model_rgb.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
    conv1_rbg = model_rgb.keras_model.get_layer("conv1")
    kernel_rgb, bias_rgb = conv1_rbg.get_weights()
    
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask", "conv1"])
    conv1_rgbd = model.keras_model.get_layer("conv1")
    kernel_rgbd = np.concatenate((kernel_rgb, np.mean(kernel_rgb, keepdims=True, axis=2)), axis=2)
    conv1_rgbd.set_weights([kernel_rgbd, bias_rgb])

elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)
elif init_with == "last_no_bbox":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask", "conv1"])
elif init_with == "last_no_top":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
if init_with != "last":
    # Train the head branches
    # Passing layers="heads" freezes all layers except the head
    # layers. You can also pass a regular expression to select
    # which layers to train by name pattern.
    model.train(dataset_train, dataset_val, 
                learning_rate=config.LEARNING_RATE, 
                epochs=1, 
                layers='heads')

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
import imgaug
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=40, # THIS IS THE FINAL NUMBER OF EPOCHS, meaning if val =20 and we restart from 15 then only 5 epochs will be run !!!
            layers="all",
            augmentation = imgaug.augmenters.Sequential([ 
                imgaug.augmenters.Fliplr(0.5), 
                imgaug.augmenters.Flipud(0.5)])
            )

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_hands_only.h5")
model.keras_model.save_weights(model_path)

## Detection

In [ ]:
class InferenceConfig(HandsConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    USE_MINI_MASK = False

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = os.path.join(MODEL_DIR, "mask_rcnn_hands_only_orig.h5")
#model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image from dataset
# image_id = random.choice(dataset_val.image_ids)
# original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#     modellib.load_image_gt(dataset_val, inference_config, 
#                            image_id)

# log("original_image", original_image)
# log("image_meta", image_meta)
# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)

# visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
#                             dataset_train.class_names, figsize=(8, 8))
#                             results = model.detect([original_image], verbose=1)

# r = results[0]
# visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
#                             dataset_val.class_names, r['scores'], ax=get_ax())

Test on real image

In [ ]:
from mrcnn.utils import resize
from importlib import reload
reload(visualize)
def evaluate_image(dataset_path, image_name, open_cv=True):
    image_depth = skimage.io.imread(os.path.join(dataset_path, "Depth", image_name))
    image_depth = resize(image_depth, (config.IMAGE_MIN_DIM, config.IMAGE_MAX_DIM), preserve_range=True)
    image_depth = image_depth[..., np.newaxis]

    image_rgb = skimage.io.imread(os.path.join(dataset_path, "RGB", image_name))
    image_rgb = resize(image_rgb, (config.IMAGE_MIN_DIM, config.IMAGE_MAX_DIM), preserve_range=True)
    image_full = np.concatenate([image_rgb, image_depth], axis=2)
            
    r = model.detect([image_full], verbose=0)[0]
    visualize.display_instances(image_full, r['rois'], r['masks'], r['class_ids'], 
                        dataset_val.class_names, r['scores'],title=image_name, ax=get_ax(), min_score = 0.1)
    
test_images = ["342", "341", "355", "374", "417", "452", "488", "503", "517", "525"]
real_imgs_path = os.path.join("C:/", "datasets", "camera_image", "2_ruce_rukavice_250")


for img in test_images:
    evaluate_image(real_imgs_path, img+".png")

In [ ]:
# #x
files = list(os.walk(os.path.join(real_imgs_path, "Depth")))[0][2]
for f in files:
    evaluate_image(real_imgs_path, f, open_cv=True)


In [ ]:

# for img in range(1,6):
#     evaluate_image(real_imgs_path, str(img)+".png")

## Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))